In [17]:
from quimb.tensor.fermion.block_interface import set_options
from quimb.tensor.fermion.fermion_core import FermionTensor, FermionTensorNetwork
set_options(use_cpp=False, symmetry="Z2")

In [18]:
from quimb.tensor.fermion.block_gen import rand_single_block

In [19]:
shape = (2, 3, 5) # (a, b, c)
pattern = "+--" # corresponds to S_a - S_b - S_c = Net symmetry
dq = 1 # Net symmetry being Z2(1)
ind = 2 # allow last index to take the net symmetry

data = rand_single_block(shape, pattern=pattern, dq=dq, ind=ind)

print("Symmetry Pattern:", data.pattern)
print("Net Symmetry: ", data.dq)

'''
if Python backend is used, we can print out the symmetry informations
'''
if hasattr(data, "blocks"):
    for iblk in data.blocks:
        print(iblk.q_labels, iblk.shape) # Z2(0)-Z2(0)-Z2(1) = Net Symmetry Z2(1)

T = FermionTensor(data, inds=('a','b','c'))

Symmetry Pattern: +--
Net Symmetry:  < N=1 >
(< N=0 >, < N=0 >, < N=1 >) (2, 3, 5)


In [20]:
from quimb.tensor.fermion.block_gen import rand_all_blocks
shape = (2, 3, 5)
symmetry_info = ((0,1), (0,1), (0,))
# allow S_a/S_b to be Z2(0) and Z2(1) only Z2(0) for S_c
pattern = "+--"
dq = 1

data = rand_all_blocks(shape, symmetry_info, pattern=pattern, dq=dq)
if hasattr(data, "blocks"):
    for iblk in data.blocks:
        print(iblk.q_labels, iblk.shape)

T = FermionTensor(data, inds=('a','b','c'))
print(T)

(< N=0 >, < N=1 >, < N=0 >) (2, 3, 5)
(< N=1 >, < N=0 >, < N=0 >) (2, 3, 5)
FermionTensor(shape=(4, 6, 5), inds=('a', 'b', 'c'), tags=oset([]), backend='pyblock3', dtype='float64')


In [21]:
import numpy as np
shape = (1,1,1)
data1 = rand_all_blocks(shape, symmetry_info, pattern=pattern, dq=dq)
T1 = FermionTensor(data1, inds=('a','b', 'c'))

data2 = rand_all_blocks(shape, symmetry_info, pattern=pattern, dq=dq)
T2 = FermionTensor(data2, inds=('c','d', 'e'))

from quimb.tensor.tensor_core import tensor_contract

out = tensor_contract(T1, T2, output_inds=('a', 'b', 'd', 'e'))  # \hat{T}_1 \hat{T}_2
out1 = tensor_contract(T2, T1, output_inds=('a', 'b', 'd', 'e')) # \hat{T}_2 \hat{T}_1

# note contraction has an order in Fermion Tensors
# T1 \dot T2 != T2 \dot T1
print("T1 @ T2")
for iblk in out.data:
    print(np.asarray(iblk).ravel(), iblk.q_labels)

print("T2 @ T1")
for iblk in out1.data:
    print(np.asarray(iblk).ravel(), iblk.q_labels)


T1 @ T2
[0.02132175] (< N=0 >, < N=1 >, < N=1 >, < N=0 >)
[0.00144821] (< N=1 >, < N=0 >, < N=1 >, < N=0 >)
T2 @ T1
[-0.02132175] (< N=0 >, < N=1 >, < N=1 >, < N=0 >)
[-0.00144821] (< N=1 >, < N=0 >, < N=1 >, < N=0 >)


In [22]:
tn = T1 & T2 # equivalent to \hat{T2} \hat{T1}
out = tn.contract(all)
for iblk in out.data:
    print(np.asarray(iblk).ravel(), iblk.q_labels)

from quimb.tensor.fermion.fermion_core import FermionTensorNetwork
tn = FermionTensorNetwork((T1, T2), virtual=True) # T_2 T_1
out = tn.contract(all)
for iblk in out.data:
    print(np.asarray(iblk).ravel(), iblk.q_labels)


[0.02132175] (< N=1 >, < N=0 >, < N=0 >, < N=1 >)
[0.00144821] (< N=1 >, < N=0 >, < N=1 >, < N=0 >)
[0.02132175] (< N=1 >, < N=0 >, < N=0 >, < N=1 >)
[0.00144821] (< N=1 >, < N=0 >, < N=1 >, < N=0 >)


In [23]:
from quimb.tensor.fermion.block_gen import gen_2d_bonds
from quimb.tensor.fermion.fermion_2d import FPEPS
Lx, Ly = (4, 3)
z2arr = np.ones((Lx, Ly)) # net Z2(1) at each site
physical_infos = dict()
for ix in range(Lx):
    for iy in range(Ly):
        physical_infos[ix, iy] = (0,1) # all physical legs allow Z2(0) and Z2(1) symmetry        
symmetry_infos, dq_infos = gen_2d_bonds(z2arr, physical_infos)

bond_dim = 2
peps = FPEPS.rand(Lx, Ly, bond_dim, symmetry_infos, dq_infos, phys_dim=2)
peps.normalize_()

<FPEPS(tensors=12, indices=29, Lx=4, Ly=3, max_bond=4)>

In [24]:
from quimb.tensor.fermion.fermion_2d_tebd import Hubbard2D
t = 1
u = 4
mu = 0
ham = Hubbard2D(t, u, Lx, Ly, mu=mu)
ene = peps.compute_local_expectation(ham.terms, max_bond=32)
print(ene)

6.800168871392267


In [25]:
from quimb.tensor.fermion.block_interface import ParticleNumber, measure_SZ
pnterms = dict()
pnop = ParticleNumber()
szterms = dict()
szop = measure_SZ()
for ix in range(Lx):
    for iy in range(Ly):
        pnterms[ix,iy]=pnop
        szterms[ix,iy]=szop
pntot = peps.compute_local_expectation(pnterms, max_bond=32)
sztot = peps.compute_local_expectation(szterms, max_bond=32)


In [26]:
print(pntot)
print(sztot)

11.93751858696992
0.2903180199133577


In [27]:
from quimb.tensor.fermion.fermion_2d_tebd import SimpleUpdate

su = SimpleUpdate(
    psi0=peps,
    ham=ham,
    D=6,
    chi=32,
    compute_energy_every=10,
    compute_energy_per_site=True,
    ordering = 'random',
    keep_best=True,
)
su.evolve(10, tau=0.01)



  0%|          | 0/10 [00:02<?, ?it/s]


AttributeError: 'LocalHam2D' object has no attribute 'nsites'

In [ ]:
psi = su.get_state()
pntot1 = psi.compute_local_expectation(pnterms, max_bond=32, normalized=True)
sztot1 = psi.compute_local_expectation(szterms, max_bond=32, normalized=True)
print("Before Simple Update")
print("N=%.6f, Sz=%.6f"%(pntot, sztot))
print("After Simple Update")
print("N=%.6f, Sz=%.6f"%(pntot1, sztot1))